<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/9_GLM_California_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de regresión GLM usando la base de datos de precios de casas de California

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.genmod.generalized_linear_model as glm
from sklearn.metrics import mean_squared_error

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

In [ ]:
df = pd.read_feather('cdiae/data/03_model_input/california_housing_clean.ftr')
df.head()

Seleccionamos las variables relevantes para el modelo (las cuales hemos reducido a 6 en las tareas de analisis bivariantes y de componentes principales anteriores)

In [ ]:
df_model = df[['Longitude','MedInc','Population','AveOccup','AveBedrms','target']]

In [ ]:
df_model.describe()

Especificamos la fórmula del GLM

In [ ]:
formula = 'target ~ Longitude + MedInc + Population + AveOccup + AveBedrms'

Ajustamos GLM con familia Poisson

In [ ]:
model = smf.glm(formula=formula, data=df_model, family=sm.families.Poisson())
results = model.fit()

Resumen del modelo

In [ ]:
print(results.summary())

Predicciones sobre el conjunto completo:

In [ ]:
df['Estimacion_GLM'] = results.predict(df)
df['Error_GLM'] = results.resid_response
df.head()

Calculamos el RMSE (este indicador sirve para compararse con otras regresiones)

In [ ]:
rmse = np.sqrt(mean_squared_error(df.target, df.Estimacion_GLM))
rmse

Graficamos los errores y comprobamos que no siguen estrictamente una distribución normal con media cero

In [ ]:
df.Error_GLM.hist(bins=50)

Calculamos otros estadísticos de la regresión:

In [ ]:
print(f'RMSE: {rmse:.4f}')
print(f'AIC: {results.aic:.2f}')
print(f'BIC: {results.bic_llf:.2f}')